# Jaimie Cairns
# Final Project Part 3

# Title: ILEARN Prediction App 

# Importing the Data

This data is stored in multiple Excel files, so there are several steps to import all of the relevant Excel sheets and combine them.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import warnings
warnings.filterwarnings("ignore")
import streamlit as st

In [2]:
years=['2024'] #to shorten coding

levels=["Corporation"]
sub_levels=["Corporation","Corporation-FRL-SE-ELL-Disaggregated","Corporation-Gender-and-Ethnicity-Disaggregated"]

In [3]:
#create dictionary to use to store dataframes
dic={year:{level:{sub_level:[] for sub_level in sub_levels} for level in levels}for year in years}
dic

{'2024': {'Corporation': {'Corporation': [],
   'Corporation-FRL-SE-ELL-Disaggregated': [],
   'Corporation-Gender-and-Ethnicity-Disaggregated': []}}}

In [5]:
#dic={}
#https://www.in.gov/doe/files/ILEARN-2023-Grade3-8-Final-School.xlsx

base="https://www.in.gov/doe/files/ILEARN-"
base2="-Grade3-8-Final-"
divider="-"
ending=".xlsx"

for year in years:
    print(year)
    for sub_level in sub_levels:
        print(sub_level)
        link=base+year+base2+sub_level+ending
        sheets= pd.ExcelFile(link).sheet_names #["ELA & Math"] 
        sheets_dic={sheet:[]for sheet in sheets}
        for sheet in sheets:

            #create temporary df to count blank cells to determine length to skip rows
            if sheet.startswith("ELA & Math"):
                df=pd.read_excel(link, sheet_name=sheet)
                a=df.isnull().iloc[:10,0]
                del df
                count=1
                for value in a:
                    if value == False:
                        count+=1
                    if value == True:
                        break

                sheets_dic[sheet]= pd.read_excel(link, 
                           skiprows=count,
                        header=[0,1], 
                           #index_col=[0,1], 
                           sheet_name=sheet)
                print("completed ", sheet)
                dic[year][sub_level]={key:sheets_dic[key] for key in sheets_dic.keys()}




2024
Corporation
completed  ELA & Math
Corporation-FRL-SE-ELL-Disaggregated
completed  ELA & Math Socio Economic
completed  ELA & Math Special Education
completed  ELA & Math English Learners
Corporation-Gender-and-Ethnicity-Disaggregated
completed  ELA & Math Ethnicity
completed  ELA & Math Gender


In [6]:
#Create a copy of the dictionary to avoid needing to import the data if a mistake is made
import copy
copy_dic=copy.deepcopy(dic)


In [7]:
copy_dic['2024']['Corporation']['ELA & Math'].head()

Unnamed: 0_level_0                Unnamed: 1_level_0  \
             Corp ID                         Corp Name   
0                 15   Adams Central Community Schools   
1                 25     North Adams Community Schools   
2                 35               South Adams Schools   
3                125  MSD Southwest Allen County Schls   
4                225    Northwest Allen County Schools   

                        Grade 3                            \
  ELA & Math\nTotal\nProficient ELA & Math\nTotal\nTested   
0                            30                      89.0   
1                            21                     100.0   
2                            30                      90.0   
3                           267                     546.0   
4                           259                     592.0   

                                                   Grade 4  \
  ELA & Math\nProficient \n% ELA & Math\nTotal\nProficient   
0                   0.337079                            38   
1                       0.21                            28   
2                   0.333333                            34   
3                   0.489011                           249   
4                     0.4375                           256   

                                                        \
  ELA & Math\nTotal\nTested ELA & Math\nProficient \n%   
0                      94.0                   0.404255   
1                      90.0                   0.311111   
2                     100.0                       0.34   
3                     538.0                   0.462825   
4                     654.0                   0.391437   

                        Grade 5                            ...  \
  ELA & Math\nTotal\nProficient ELA & Math\nTotal\nTested  ...   
0                            25                     102.0  ...   
1                            15                      82.0  ...   
2                            36                      90.0  ...   
3                           245                     569.0  ...   
4                           219                     650.0  ...   

                     Grade 6                       Grade 7  \
  ELA & Math\nProficient \n% ELA & Math\nTotal\nProficient   
0                   0.567568                            46   
1                   0.304878                            21   
2                   0.247525                            29   
3                   0.384095                           250   
4                   0.327007                           210   

                                                        \
  ELA & Math\nTotal\nTested ELA & Math\nProficient \n%   
0                     100.0                       0.46   
1                      93.0                   0.225806   
2                      93.0                   0.311828   
3                     623.0                   0.401284   
4                     666.0                   0.315315   

                        Grade 8                            \
  ELA & Math\nTotal\nProficient ELA & Math\nTotal\nTested   
0                            48                      99.0   
1                            26                      98.0   
2                            42                      88.0   
3                           231                     612.0   
4                           203                     651.0   

                                         Corporation Total  \
  ELA & Math\nProficient \n% ELA & Math\nTotal\nProficient   
0                   0.484848                           250   
1                   0.265306                           136   
2                   0.477273                           196   
3                   0.377451                          1469   
4                   0.311828                          1371   

                                                        
  ELA & Math\nTotal\nTested ELA & Math\nProficient \n%  
0                       595                

In [8]:
#Delete empty dictionaries (dictionaries that were not imported because they contained data other than 
#the ELA & Math data)
for level in list(copy_dic):
    for sub_level in list(copy_dic[level]):
        for sheet in list(copy_dic[level][sub_level]):
            #print(level,sub_level,sheet)
            if not sheet.startswith('ELA & Math'):
                del copy_dic[level][sub_level][sheet]

In [9]:
import re
#Fix the column names to delete the split lines (Only run this once)
for level in copy_dic:
    for sub_level in copy_dic[level]:
        for sheet in copy_dic[level][sub_level]:
            copy_dic[level][sub_level][sheet].columns=copy_dic[level][sub_level][sheet].columns.map('|'.join).str.strip('|')
            col1_names=list(copy_dic[level][sub_level][sheet].columns)

            col1_new_names=[]
            for name in col1_names:
                name =" ".join(name.splitlines())
                name=re.sub(r'Unn.+?(?=\|)\|','',name)
                col1_new_names.append(name)

            name_copy_dict={}
            for name in col1_names:
                for value in col1_new_names:
                    name_copy_dict[name]=value
                    col1_new_names.remove(value)
                    break
            
            copy_dic[level][sub_level][sheet]=copy_dic[level][sub_level][sheet].rename(columns=name_copy_dict)
            

In [10]:
#Drop columns containing individual grade information
def drop_grade_cols(df):
    temp_col=df.columns
    drop_col=['Corp ID']
    for col in temp_col:
        if col.startswith("Grade"):
            drop_col.append(col)

    for col in drop_col:
        df.drop([col],axis=1,inplace=True)
    return(df)


for level in copy_dic:
    for sub_level in copy_dic[level]:
        for sheet in copy_dic[level][sub_level]:
            copy_dic[level][sub_level][sheet]=drop_grade_cols(copy_dic[level][sub_level][sheet])


In [11]:
# set index to Corp ID
for level in copy_dic:
    for sub_level in copy_dic[level]:
        for sheet in copy_dic[level][sub_level]:
            copy_dic[level][sub_level][sheet]=copy_dic[level][sub_level][sheet].set_index('Corp Name')

In [12]:
#The NAs are *** in this spreadsheet so we need to convert those to 0 

for level in copy_dic:
    for sub_level in copy_dic[level]:
        for sheet in copy_dic[level][sub_level]:
            copy_dic[level][sub_level][sheet].replace('***',np.nan,inplace=True)


In [13]:
for level in copy_dic:
    for sub_level in copy_dic[level]:
        for sheet in copy_dic[level][sub_level]:
            copy_dic[level][sub_level][sheet]=copy_dic[level][sub_level][sheet].astype(float)


In [14]:
#Add Above State to each year
for level in copy_dic: 

    yravg=copy_dic[level]['Corporation']['ELA & Math']['Corporation Total|ELA & Math Total Proficient'].sum()/copy_dic[level]['Corporation']['ELA & Math']['Corporation Total|ELA & Math Total Tested'].sum()
    copy_dic[level]['Corporation']['ELA & Math']=copy_dic[level]['Corporation']['ELA & Math'].assign(AboveState=lambda x: copy_dic[level]['Corporation']['ELA & Math']['Corporation Total|ELA & Math Proficient  %']>= yravg)


In [15]:
copy_dic['2024']['Corporation']['ELA & Math'].head()

,Corporation Total|ELA & Math Total Proficient,Corporation Total|ELA & Math Total Tested,Corporation Total|ELA & Math Proficient %,AboveState
Corp Name,,,,
Adams Central Community Schools,250.0,595.0,0.420168,True
North Adams Community Schools,136.0,545.0,0.249541,False
South Adams Schools,196.0,562.0,0.348754,True
MSD Southwest Allen County Schls,1469.0,3479.0,0.422248,True
Northwest Allen County Schools,1371.0,3898.0,0.351719,True


In [16]:
#Create Single Dataframe from Dictionaries
count=0
for year in copy_dic:
    for sub_level in copy_dic[year]:
        for sheet in copy_dic[year][sub_level]:
            if count==0:
                df=copy_dic[year][sub_level][sheet]
                count+=1
            else:
                tempdf=copy_dic[year][sub_level][sheet]
                df=df.join(tempdf)
            

In [17]:
#add school type to dataframe
directory=pd.read_excel('Directory.xlsx')
directory=directory.rename(columns={"CORPORATION_NAME":"Corp Name"})
corp_type=directory[['Corp Name','CORPORATION_TYPE']]
corp_type.set_index('Corp Name')
df=pd.merge(df,corp_type,on="Corp Name",how="inner")

In [18]:
df.head()

,Corp Name,Corporation Total|ELA & Math Total Proficient,Corporation Total|ELA & Math Total Tested,Corporation Total|ELA & Math Proficient %,AboveState,Paid Meals|ELA & Math Total Proficient,Paid Meals|ELA & Math Total Tested,Paid Meals|ELA & Math Proficient %,Free/Reduced Price Meals|ELA & Math Total Proficient,Free/Reduced Price Meals|ELA & Math Total Tested,...,White|ELA & Math Total Proficient,White|ELA & Math Total Tested,White|ELA & Math Proficient %,Female|ELA & Math Total Proficient,Female|ELA & Math Total Tested,Female|ELA & Math Proficient %,Male|ELA & Math Total Proficient,Male|ELA & Math Total Tested,Male|ELA & Math Proficient %,CORPORATION_TYPE
0,Adams Central Community Schools,250.0,595.0,0.420168,True,206.0,423.0,0.486998,44.0,172.0,...,232.0,544.0,0.426471,125.0,276.0,0.452899,125.0,319.0,0.391850,School Corporation
1,North Adams Community Schools,136.0,545.0,0.249541,False,90.0,281.0,0.320285,46.0,264.0,...,114.0,426.0,0.267606,63.0,271.0,0.232472,73.0,274.0,0.266423,School Corporation
2,South Adams Schools,196.0,562.0,0.348754,True,125.0,291.0,0.429553,71.0,271.0,...,178.0,487.0,0.365503,101.0,280.0,0.360714,95.0,282.0,0.336879,School Corporation
3,MSD Southwest Allen County Schls,1469.0,3479.0,0.422248,True,1289.0,2652.0,0.486048,180.0,827.0,...,1199.0,2589.0,0.463113,665.0,1620.0,0.410494,804.0,1859.0,0.432491,School Corporation
4,Northwest Allen County Schools,1371.0,3898.0,0.351719,True,1159.0,2947.0,0.393281,212.0,951.0,...,1115.0,2995.0,0.372287,640.0,1856.0,0.344828,731.0,2042.0,0.357982,School Corporation


In [ ]:
#run to upload prior csv file
#df=pd.read_csv('df2024.csv',index_col=0)

In [24]:
def year_average(df):
    yr_avg=(df['Corporation Total|ELA & Math Total Proficient'].sum())/(df['Corporation Total|ELA & Math Total Tested'].sum())
    return yr_avg



df=df.fillna(0)
df=df.assign(AboveState=lambda x: df['Corporation Total|ELA & Math Proficient  %']>= year_average(df))


In [26]:
#Calculate percentage of black and white students
df=df.assign(BlackPopulation=lambda x:df['Black|ELA & Math Total Tested']/
                     df['Corporation Total|ELA & Math Total Tested'])

df=df.assign(WhitePopulation=lambda x:df['White|ELA & Math Total Tested']/
                     df['Corporation Total|ELA & Math Total Tested'])
#Calculate percentage of Non-English Language Learners
df=df.assign(Non_ENL=lambda x:df['Non-English Language Learners|ELA & Math Total Tested']/
                     df['Corporation Total|ELA & Math Total Tested'])
#Calculate percentage of Students Pay for Lunch 
df=df.assign(Paid_Lunch=lambda x:df['Paid Meals|ELA & Math Total Tested']/
                     df['Corporation Total|ELA & Math Total Tested'])
#Calculate Percentage of Students in Gen Ed Population

df=df.assign(Gen_Ed=lambda x:df['General Education|ELA & Math Total Tested']/
                     df['Corporation Total|ELA & Math Total Tested'])


corp_to_numbers={
    "School Corporation":0,
    'Charter':1,
    'Non-Public Organization':2,
    'Special Education Co-op':2,
    'State Run':2,
    'University School':2
}

df['Corp Type']=df['CORPORATION_TYPE'].map(corp_to_numbers)

In [27]:
df['AboveState']=df['AboveState'].astype(int)
df_model=df[['AboveState',

                     'WhitePopulation', 
                     'Non_ENL', 
                     'Paid_Lunch', 
                     'Corp Type'
                    ]]

In [28]:
df_model.to_csv('df_model.csv',index=False)

# Personal Contribution Statement

Because there is only one member of my team, I am the only one that contributed to all of the parts of this project. 